In [2]:
import matplotlib.pyplot as plt
import numpy as np
from subprocess import call
import re
from collections import OrderedDict
import math
from datetime import datetime

def to_microsecond(str):
    t = datetime.strptime(str, "%H:%M:%S.%f")
    return t.microsecond + 1000000 * (t.second + 60 * (t.minute + t.hour * 60))

dirname = '/home/heyihong/SystemMeasurement/Network/'

In [27]:
def analyze_rtt_data(filename):
    lines = open(dirname + filename).readlines()
    rtts = []
    for idx in range(0, len(lines) - 1):
        sp1 = lines[idx].split()
        sp2 = lines[idx + 1].split()
        if sp1[6] == '[P.],' and sp2[6] == '[.],':
            rtts.append(to_microsecond(sp2[0]) - to_microsecond(sp1[0]))
    print filename, ' avg (microseconds), standard variance:', np.average(rtts), math.sqrt(np.var(rtts))

analyze_rtt_data('remote-rtt.result')
analyze_rtt_data('local-rtt.result')

remote-rtt.result  avg (microseconds), standard variance: 338.6385518 99.2573462993
local-rtt.result  avg (microseconds), standard variance: 28.5299 95.0668622917


In [12]:
def analyze_conn_data(filename):
    data = []
    for line in open(dirname + filename).readlines():
        sp = line.split()
        tup = (to_microsecond(sp[0]), sp[6])
        data.append(tup)
    setup = []
    teardown = []
    for idx in range(0, len(data) - 2):
        if data[idx][1] == '[S],' and data[idx + 1][1] == '[S.],' and data[idx + 2][1] == '[.],':
            setup.append(data[idx + 2][0] - data[idx][0])
        if data[idx][1] == '[F.],' and data[idx + 1][1] == '[F.],' and data[idx + 2][1] == '[.],':
            teardown.append(data[idx + 2][0] - data[idx][0])
    print filename, 'setup avg (microseconds), standard variance:', np.average(setup), math.sqrt(np.var(setup))
    print filename, 'teardown avg (microseconds), standard variance:', np.average(teardown), math.sqrt(np.var(teardown))

analyze_conn_data('remote-conn.result')
analyze_conn_data('local-conn.result')

remote-conn.result setup avg (microseconds), standard variance: 353.038976857 30.6209141099
remote-conn.result teardown avg (microseconds), standard variance: 361.045622688 42.7678231601
local-conn.result setup avg (microseconds), standard variance: 28.956 4.19095025024
local-conn.result teardown avg (microseconds), standard variance: 42.2517552658 12.3115609501


In [25]:
def analyze_peak_data(filename):
    # gap is equal to 1 second
    gap = 1000000
    data = []
    lines = open(dirname + filename).readlines()
    for idx in range(len(lines)):
        sp = lines[idx].split()
        if sp[7] == 'ack':
            (t, host1, host2, ack) = (to_microsecond(sp[0]), sp[2], sp[4][:-1], int(sp[8][:-1]))
            data.append(((host1, host2), t / gap, ack))
    peaks = []
    peak = prev_ack = ack = 0
    for idx in range(len(data)):
        ack = max(ack, data[idx][2])
        peak = max(peak, ack - prev_ack)
        if idx != len(data) -1 and data[idx + 1][1] != data[idx][1]:
            prev_ack = ack
        if idx == len(data) - 1 or data[idx + 1][0] != data[idx][0]:
            peaks.append(peak)
            # the next iteration starts
            peak = prev_ack = ack = 0
    print filename, 'avg (bytes), standard variance:', np.average(peaks), math.sqrt(np.var(peaks))

analyze_peak_data('remote-peak.result')
analyze_peak_data('local-peak.result')

remote-peak.result avg (bytes), standard variance: 111000784.0 0.0
local-peak.result avg (bytes), standard variance: 2703793070.0 0.0
